## A jupyter notebook version file for the `main.py`


Set `autoreload` to execute the change in `.py` files.


In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import _pickle
import natsort
from pathlib import Path
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from __utils__ import image_processing
from __utils__ import label_processing
from __utils__ import labeling
from __utils__ import loso_preparing
from __utils__ import functions

In [17]:
dataset_dir = "D:/Databases/CAS(ME)^2"
# dataset_dir = "F:/HEH/Databases/CAS(ME)^2"
# dataset_dir = "/data/disk1/heh/databases/CAS(ME)^2"

images_loading = False
image_size = 128
load_cropped_images = False
expression_type = "mae"  # macro-expression spotting
# expression_type = "me" # micro-expression spotting
debug_preds = True
labeling_function = "pseudo_labeling"
# labeling_function = "original_labeling"
model_names = {
    0: "SOFTNet",
    1: "SOFTNetCBAM",
    2: "ViT-B",
    3: "SL-ViT-B",
    4: "Swin-T",
    5: "Swin-S",
    6: "L-Swin-T",
    7: "S-Swin-T",
    8: "SL-Swin-T",
    9: "SL-Swin-S",
}
model_name = model_names[8]
batch_size = 48
epochs = 25
save_preds = False
preds_stem = (
    f"{expression_type}_"
    + model_name.lower().replace("-", "_")
    + f"_batch_size_{batch_size}"
    + f"_epochs_{epochs}"
    + f"_{labeling_function}"
    + f"_{image_size}"
    # + "_3"
)
preds_path = Path(dataset_dir, "preds", preds_stem).with_suffix(".pkl")
print(f"preds_path: {preds_path}")

preds_path: D:\Databases\CAS(ME)^2\preds\mae_sl_swin_t_batch_size_48_epochs_25_pseudo_labeling_128.pkl


## Load Images


When debug the image processing, the videos_images is from cropped_rawpic, whereas the other variables are from rawpic.


In [18]:
videos_images, subjects, subjects_videos_code = image_processing.load_images(
    dataset_dir,
    images_loading=images_loading,
    image_size=image_size,
    load_cropped_images=load_cropped_images,
)

subject:  s15
subject:  s16
subject:  s19
subject:  s20
subject:  s21
subject:  s22
subject:  s23
subject:  s24
subject:  s25
subject:  s26
subject:  s27
subject:  s29
subject:  s30
subject:  s31
subject:  s32
subject:  s33
subject:  s34
subject:  s35
subject:  s36
subject:  s37
subject:  s38
subject:  s40


In [19]:
print("subjects:", subjects)
print("subjects_videos_code:", subjects_videos_code)

subjects: ['s15', 's16', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's40']
subjects_videos_code: [['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0508'], ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ['0102', '0402', '0505', '0507', '0502'], ['0502'], ['0101', '0401'], ['0101', '0102', '0402', '0503', '0508'], ['0102', '0402', '0503', '0507'], ['0101', '0401', '0402', '0502', '0507'], ['0101', '0102', '0502', '0508'], ['0101', '0102', '0401', '0503'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0502'], ['0101', '0102', '0401', '0502', '0503', '0505', '0507'], ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0102', '0402'], ['0401', '0402', '0503'], ['0102'], ['0401', '0505'], ['0101', '0402', '0502', '0505', '0507', '0508'], ['0502', '0507'], ['0401', '0502', '

## Load Excel 


In [20]:
Excel_data = label_processing.load_excel(dataset_dir)
Excel_data.head(5)

,participant,video_name_&_expression_number,onset,apex,offset,AUs,extimated_emotion,expression_type,self-reported_emotion,video_name,video_code,subject
0,1,anger1_1,557,572,608,4+10+14+15,negative,macro-expression,anger,anger1,0401,s15
1,1,anger1_2,2854,2862,2871,38,others,macro-expression,sadness,anger1,0401,s15
2,1,anger2_1,2155,2163,0,NaN,negative,macro-expression,anger,anger2,0402,s15
3,1,anger2_2,3363,3371,3383,4+7+14,negative,macro-expression,anger,anger2,0402,s15
4,1,anger2_3,3380,3386,3407,4+14+38,negative,macro-expression,anger,anger2,0402,s15


## Load Ground Truth Labels


In [21]:
(
    clean_videos_images,
    clean_subjects_videos_code,
    clean_subjects,
    clean_subjects_videos_ground_truth_labels,
) = label_processing.load_ground_truth_labels(
    dataset_dir,
    expression_type,
    videos_images,
    subjects_videos_code,
    subjects,
    Excel_data,
)

required_videos_index:  [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]
len(clean_videos_images) = 88


In [22]:
print("len(clean_subjects): ", len(clean_subjects))
print("clean_subjects: ", clean_subjects)
print("len(clean_subjects_videos_code): ", len(clean_subjects_videos_code))
print("clean_subjects_videos_codes: ", clean_subjects_videos_code)
print(
    "len(clean_subjects_videos_ground_truth_labels): ",
    len(clean_subjects_videos_ground_truth_labels),
)
print(
    "clean_subjects_videos_ground_truth_labels: ",
    clean_subjects_videos_ground_truth_labels,
)

len(clean_subjects):  20
clean_subjects:  ['s15' 's16' 's19' 's20' 's21' 's22' 's23' 's24' 's25' 's26' 's27' 's30'
 's31' 's32' 's33' 's34' 's36' 's37' 's38' 's40']
len(clean_subjects_videos_code):  20
clean_subjects_videos_codes:  [['0101', '0102', '0401', '0402', '0502', '0503', '0505'], ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ['0102', '0505', '0507'], ['0502'], ['0101', '0401'], ['0101', '0102', '0402', '0503', '0508'], ['0102', '0402', '0507'], ['0101', '0401', '0402', '0502', '0507'], ['0102', '0502', '0508'], ['0101', '0102', '0401', '0503'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0102', '0401', '0502', '0503', '0505', '0507'], ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507'], ['0102', '0402'], ['0401', '0402', '0503'], ['0401', '0505'], ['0101', '0402', '0502', '0505', '0507', '0508'], ['0502', '0507'], ['0401', '0502', '0503']]
len(clean_subjects_vide

## Calculate `k`


In [23]:
k = label_processing.calculate_k(clean_subjects_videos_ground_truth_labels)

k (Half of average length of expression) =  18


## Labeling


In [24]:
if labeling_function == "pseudo_labeling":
    labels = labeling.get_pseudo_labels(
        clean_videos_images, clean_subjects_videos_ground_truth_labels, k
    )
elif labeling_function == "original_labeling":
    labels = labeling.get_original_labels(
        clean_videos_images, clean_subjects_videos_ground_truth_labels, k
    )

Total frames: 231999


## Prepare for LOSO


In [25]:
y, groups = loso_preparing.prepare_for_loso(
    labels,
    clean_subjects,
    clean_videos_images,
    clean_subjects_videos_ground_truth_labels,
    k,
)

Frame Index for each subject:-

subject s15 ( group = 0): 0 -> 18470
subject s15 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  7

subject s16 ( group = 1): 18470 -> 37392
subject s16 has 7 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  14

subject s19 ( group = 2): 37392 -> 43977
subject s19 has 3 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  17

subject s20 ( group = 3): 43977 -> 46233
subject s20 has 1 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  18

subject s21 ( group = 4): 46233 -> 51966
subject s21 has 2 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  20

subject s22 ( group = 5): 51966 -> 62840
subject s22 has 5 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  25

subject s23 ( group = 6): 62840 -> 71496
subject s23 has 3 clean video(s)
sum clean_subject_videos_ground_truth_labels_len:  28

subject s24 ( group = 7): 71496 -> 87084
subject s24 has 5 clean video

## Training


In [26]:
if debug_preds is False:
    from __utils__.training_dev import train

    preds = train(
        dataset_dir=dataset_dir,
        clean_subjects=clean_subjects,
        image_size=image_size,
        y=y,
        expression_type=expression_type,
        model_name=model_name,
        train_or_not=True,
        batch_size=batch_size,
        epochs=epochs,
        preds_path=preds_path,
    )
else:
    with open(preds_path, "rb") as pkl_file:
        preds = _pickle.load(pkl_file)
        pkl_file.close()

In [27]:
if save_preds is True:
    with open(preds_path, "wb") as pkl_file:
        _pickle.dump(preds, pkl_file)
        pkl_file.close()

## Spotting and Evaluation


In [28]:
metric_fn, result_dict = functions.spot_and_evaluate(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
    p=0.60,
    show_plot_or_not=False,
)

Split 1/20 is in process.
0 video(s) have been processed.
The current video be processed: subject s15, video 0101
The current video be processed: subject s15, video 0102
The current video be processed: subject s15, video 0401
The current video be processed: subject s15, video 0402
The current video be processed: subject s15, video 0502
The current video be processed: subject s15, video 0503
The current video be processed: subject s15, video 0505

True Positive: 4, False Posive: 21, False Negative: 11
Precision = 0.16000000000000023, Recall =0.2666666666666669, F1-Score = 0.20000000000000048
Split 1/20 is processed.

Split 2/20 is in process.
7 video(s) have been processed.
The current video be processed: subject s16, video 0101
The current video be processed: subject s16, video 0102
The current video be processed: subject s16, video 0401
The current video be processed: subject s16, video 0402
The current video be processed: subject s16, video 0502
The current video be processed: subjec

## Final Evaluation


In [29]:
functions.final_evaluate(metric_fn, result_dict)

True Positive: 70, False Posive: 258, False Negative: 228
Final Precision = 0.21341463414634168,
Final Recall =0.23489932885906062,
Final F1-Score = 0.22364217252396212

Highest Precision = 0.23735408560311305,
Highest Recall =0.27044025157232726,
Highest F1-Score = 0.25000000000000044


| Parameters | Value | Value | Value (tf) | Value (tf) | Value
| --- | --- | --- | --- | --- | ---
| model | 3D-CNN | SOFTNet | SOFTNet (generator) | SOFTNet (dev) | SOFTNet
| epochs | | 10 | 20 | 20 (252 m) | 30 (193 m) 
| batch_size | | 48 | 48 | 48 | 48
| learning_rate | | 0.0005 | 0.0005 | 0.0005 | 0.0005
| distance | | | k | k | 2k |
| True Positive | | 90 | 95 | 83 | 11
| False Positive | | 357 | 357 | 316 | 320 
| False Negative | | 210 | 203 | 215 | 287
| Precision | | 0.2013 | 0.2102 | 0.2080 | 0.0332
| Recall | | 0.3000 | 0.3188 | 0.2785 | 0.0369
| F1-Score | 0.2145 | 0.2410 | 0.2533 | 0.2381 | 0.0349


| Parameters | Value | Value | Value |
| --- | --- | --- | --- |
| model | SL-Swin-T | SL-Swin-T | SL-Swin-T |
| epochs | 20 ( m) | 25 ( m) | 30 ( m) |
| batch_size | 32 | 32 | 32 |
| p |  |  |  |
| True Positive |  |  |  |
| False Positive |  |  |  |
| False Negative |  |  |  |
| Precision |  |  |  |
| Recall |  |  | |
| F1-Score |  |  |  |


| Parameters | Value | Value | Value | Value
| --- | --- | --- | --- | ---
| model | SL-Swin-T | SL-Swin-T | SL-Swin-T_2 | SL-Swin-T_3
| epochs | 20 (2518 m) | 25 (2967 m) | 25 (2967 m) | 25 (3045 m)
| batch_size | 48 | 48 | 48 | 48
| p | 0.57 | 0.60 |0.60 | 0.60
| True Positive | 66 | 70 | 67 | 65
| False Positive | 279 | 258 | 255 | 261
| False Negative | 232 | 228 | 231 | 233
| Precision | 0.1913 | 0.2134 | 0.2081 | 0.1993
| Recall | 0.2215 | 0.2349 | 0.2248 | 0.2181
| F1-Score | 0.2053 | 0.2236 | 0.2161 | 0.2083


| Parameters | Value | Value | Value | Value |  Value | Value
| --- | --- | --- | --- | --- | --- | ---
| model | SL-Swin-S | SL-Swin-S | SL-Swin-S | SL-Swin-S | SL-Swin-S | SL-Swin-S
| epochs | 20 ( m) | 25 (7262 m) | 30 ( m) | 20 (3542 m) | 25 (5269 m) | 30 ( m)
| batch_size | 32 | 32 | 32 | 48 | 48 | 48
| p | 0.86 | 0.77 | | 0.59 | 0.57 | 0.55
| True Positive |  | 49 |  | 69 | 70 | 13
| False Positive |  | 152 |  | 261 | 269 | 289
| False Negative |  | 249 |  | 229 | 228 | 44
| Precision |  | 0.2437 |  | 0.2091 | 0.2064 | 0.0430
| Recall |  | 0.1644 | | 0.2315 | 0.2348 | 0.2280
| F1-Score |  | 0.1963 |  | 0.2197 | 0.2197 | 0.0724


| Parameters | Value | Value | Value | Value | Value
| --- | --- | --- | --- | --- | ---
| model | ViT-B | SL-ViT-B | Swin-T | L-Swin-T | S-Swin-T
| epochs | 25 (4768 m) | 25 (8585 m) | 25 ( m) | 25 (3624 m) | 25 (2416 m)
| batch_size | 48 | 96 | 48 | 48 | 48
| p | 0.60 | 0.70 | 0.57 | 0.51 | 0.49
| True Positive | 95 | 58 | 69 | 83 | 87 | 
| False Positive | 501 | 204 | 287 | 317 | 355 | 
| False Negative | 203 | 240 | 229 | 215 | 211 | 
| Precision | 0.1594 | 0.2214 | 0.1938 | 0.2075 | 0.1968 | 
| Recall | 0.3188 | 0.1946 | 0.2315 | 0.2785 | 0.2919 | 
| F1-Score | 0.2125 | 0.2071 | 0.2110 | 0.2378 | 0.2351 | 


Original Labeling


| Parameters | Value | Value |
| --- | --- | --- |
| model | Swin-T | SL-Swin-T |
| epochs | 25 | 25 |
| batch_size | 48 | 48 |
| p | 0.58 | 0.60 |
| True Positive | 72 | 71 |
| False Positive | 234 | 264 |
| False Negative | 226 | 227 |
| Precision | 0.2353 | 0.2119 |
| Recall | 0.2416 | 0.2383 |
| F1-Score | 0.2384 | 0.2243 |


## Ablation Study


In [30]:
ablation_dict = functions.ablation_study_p_dev(
    preds,
    clean_subjects_videos_ground_truth_labels,
    clean_videos_images,
    clean_subjects,
    clean_subjects_videos_code,
    k,
)

 p | TP | FP | FN | Precision | Recall | F1-Score
0.01 | 159 | 3149 | 139 | 0.0481 | 0.5336 | 0.0882 |
0.02 | 157 | 2853 | 141 | 0.0522 | 0.5268 | 0.0949 |
0.03 | 157 | 2626 | 141 | 0.0564 | 0.5268 | 0.1019 |
0.04 | 156 | 2383 | 142 | 0.0614 | 0.5235 | 0.1100 |
0.05 | 155 | 2213 | 143 | 0.0655 | 0.5201 | 0.1163 |
0.06 | 154 | 2051 | 144 | 0.0698 | 0.5168 | 0.1231 |
0.07 | 153 | 1905 | 145 | 0.0743 | 0.5134 | 0.1299 |
0.08 | 150 | 1766 | 148 | 0.0783 | 0.5034 | 0.1355 |
0.09 | 147 | 1667 | 151 | 0.0810 | 0.4933 | 0.1392 |
0.10 | 146 | 1585 | 152 | 0.0843 | 0.4899 | 0.1439 |
0.11 | 145 | 1496 | 153 | 0.0884 | 0.4866 | 0.1496 |
0.12 | 144 | 1426 | 154 | 0.0917 | 0.4832 | 0.1542 |
0.13 | 141 | 1348 | 157 | 0.0947 | 0.4732 | 0.1578 |
0.14 | 141 | 1276 | 157 | 0.0995 | 0.4732 | 0.1644 |
0.15 | 140 | 1219 | 158 | 0.1030 | 0.4698 | 0.1690 |
0.16 | 139 | 1153 | 159 | 0.1076 | 0.4664 | 0.1748 |
0.17 | 137 | 1101 | 161 | 0.1107 | 0.4597 | 0.1784 |
0.18 | 137 | 1048 | 161 | 0.1156 | 0.4597 | 0.184